Analisando mais a fundo para o HT, base de dados diferente porque tem HT (1 tempo)

In [6]:
import pandas as pd

# Carregar o arquivo fornecido
file_path_atualizado = 'C:\\Users\\msdof\\OneDrive\\Imagens\\Ligas\\BASE DE DADOS GRANDES\\futpython.xlsx'
dados = pd.read_excel(file_path_atualizado)

# Filtrar dados específicos para a liga "England Premier League" e a temporada "2022"
dados = dados[(dados['League'] == 'Brazil Serie A') & (dados['Season'].astype(str).str.contains('2022'))]

# Ordenar dados por data e rodada
dados = dados.sort_values(by=['Date', 'Rodada'])

def calcular_probabilidade_variacao(dados, n):
    # Verificar se houve variação nas últimas N rodadas
    condicao_varia = True
    for i in range(1, n+1):
        condicao_varia &= (dados['TotalGoals_HT'] != dados['TotalGoals_HT'].shift(i))

    dados[f'VariaUltimas{n}Rodadas'] = condicao_varia

    # Criar uma nova coluna para a rodada seguinte
    dados['TotalGoals_Proxima_Rodada'] = dados['TotalGoals_HT'].shift(-1)

    # Verificar se há variação na próxima rodada
    dados['VariaProximaRodada'] = dados['TotalGoals_HT'] != dados['TotalGoals_Proxima_Rodada']

    # Filtrar dados conforme a condição estabelecida
    dados_varia = dados[dados[f'VariaUltimas{n}Rodadas']]

    # Contar total de casos onde houve variação nas últimas N rodadas
    total_varia = dados_varia.shape[0]

    # Contar total de casos onde houve variação na próxima rodada após variação nas últimas N
    varia_na_proxima = dados_varia['VariaProximaRodada'].sum()

    # Calcular a probabilidade de variação
    probabilidade_variacao = None
    if total_varia > 0:
        probabilidade_variacao = varia_na_proxima / total_varia

    # Calcular a probabilidade de ter pelo menos 1 gol no HT
    total_jogos_ht_com_gols = dados_varia['TotalGoals_HT'][dados_varia['TotalGoals_HT'] > 0].count()
    probabilidade_gol_ht = total_jogos_ht_com_gols / total_varia if total_varia > 0 else None

    # Calcular a probabilidade de ter pelo menos 2 gols no FT
    total_jogos_ft_com_2_gols = dados_varia['TotalGoals_FT'][dados_varia['TotalGoals_FT'] >= 2].count()
    probabilidade_2_gols_ft = total_jogos_ft_com_2_gols / total_varia if total_varia > 0 else None

    # Calcular a probabilidade de ter no máximo 3 gols no FT
    total_jogos_ft_com_ate_3_gols = dados_varia['TotalGoals_FT'][dados_varia['TotalGoals_FT'] <= 3].count()
    probabilidade_ate_3_gols_ft = total_jogos_ft_com_ate_3_gols / total_varia if total_varia > 0 else None

    return probabilidade_variacao, total_varia, probabilidade_gol_ht, probabilidade_2_gols_ft, probabilidade_ate_3_gols_ft

# Calcular a probabilidade para diferentes valores de N
resultados = {}
for n in range(1, 11):
    probabilidade_variacao, total_jogos_testados, probabilidade_gol_ht, probabilidade_2_gols_ft, probabilidade_ate_3_gols_ft = calcular_probabilidade_variacao(dados, n)
    if probabilidade_variacao is not None:
        resultados[n] = (probabilidade_variacao, total_jogos_testados, probabilidade_gol_ht, probabilidade_2_gols_ft, probabilidade_ate_3_gols_ft)

# Exibir os resultados
for n, (probabilidade_variacao, total_jogos_testados, probabilidade_gol_ht, probabilidade_2_gols_ft, probabilidade_ate_3_gols_ft) in resultados.items():
    print(f"Probabilidade de variação na próxima rodada após variação nas últimas {n} rodadas: {probabilidade_variacao:.2%}")
    print(f"Total de jogos testados para {n} rodadas: {total_jogos_testados}")
    print(f"Probabilidade de ter pelo menos 1 gol no HT: {probabilidade_gol_ht:.2%}")
    print(f"Probabilidade de ter pelo menos 2 gols no FT: {probabilidade_2_gols_ft:.2%}")
    print(f"Probabilidade de ter no máximo 3 gols no FT: {probabilidade_ate_3_gols_ft:.2%}\n")

Probabilidade de variação na próxima rodada após variação nas últimas 1 rodadas: 68.24%
Total de jogos testados para 1 rodadas: 255
Probabilidade de ter pelo menos 1 gol no HT: 69.02%
Probabilidade de ter pelo menos 2 gols no FT: 69.02%
Probabilidade de ter no máximo 3 gols no FT: 74.51%

Probabilidade de variação na próxima rodada após variação nas últimas 2 rodadas: 71.14%
Total de jogos testados para 2 rodadas: 201
Probabilidade de ter pelo menos 1 gol no HT: 70.15%
Probabilidade de ter pelo menos 2 gols no FT: 73.13%
Probabilidade de ter no máximo 3 gols no FT: 71.64%

Probabilidade de variação na próxima rodada após variação nas últimas 3 rodadas: 74.10%
Total de jogos testados para 3 rodadas: 139
Probabilidade de ter pelo menos 1 gol no HT: 74.82%
Probabilidade de ter pelo menos 2 gols no FT: 77.70%
Probabilidade de ter no máximo 3 gols no FT: 65.47%

Probabilidade de variação na próxima rodada após variação nas últimas 4 rodadas: 79.59%
Total de jogos testados para 4 rodadas: 98

In [7]:
import pandas as pd

# Carregar o arquivo fornecido
file_path_atualizado = 'C:\\Users\\msdof\\OneDrive\\Imagens\\Ligas\\BASE DE DADOS GRANDES\\futpython.xlsx'

dados = pd.read_excel(file_path_atualizado)

# Filtrar dados específicos para a liga "England Premier League" e a temporada "2022"
dados = dados[(dados['League'] == 'Brazil Serie A') & (dados['Season'].astype(str).str.contains('2022'))]

#filtro das odds
dados = dados[dados['Odd_A_FT'] < 2]


# Ordenar dados por data e rodada
dados = dados.sort_values(by=['Date', 'Rodada'])

def calcular_probabilidade_variacao(dados, n):
    # Verificar se houve variação nas últimas N rodadas
    condicao_varia = True
    for i in range(1, n+1):
        condicao_varia &= (dados['TotalGoals_HT'] != dados['TotalGoals_HT'].shift(i))

    dados[f'VariaUltimas{n}Rodadas'] = condicao_varia

    # Criar uma nova coluna para a rodada seguinte
    dados['TotalGoals_Proxima_Rodada'] = dados['TotalGoals_HT'].shift(-1)

    # Verificar se há variação na próxima rodada
    dados['VariaProximaRodada'] = dados['TotalGoals_HT'] != dados['TotalGoals_Proxima_Rodada']

    # Filtrar dados conforme a condição estabelecida
    dados_varia = dados[dados[f'VariaUltimas{n}Rodadas']]

    # Contar total de casos onde houve variação nas últimas N rodadas
    total_varia = dados_varia.shape[0]

    # Contar total de casos onde houve variação na próxima rodada após variação nas últimas N
    varia_na_proxima = dados_varia['VariaProximaRodada'].sum()

    # Calcular a probabilidade
    if total_varia > 0:
        probabilidade = varia_na_proxima / total_varia
        return probabilidade, total_varia  # Retorna a probabilidade e a contagem
    else:
        return None, 0

# Calcular a probabilidade para diferentes valores de N
resultados = {}
for n in range(1, 11):
    probabilidade, total_jogos_testados = calcular_probabilidade_variacao(dados, n)
    if probabilidade is not None:
        resultados[n] = (probabilidade, total_jogos_testados)

# Exibir os resultados
for n, (probabilidade, total_jogos_testados) in resultados.items():
    print(f"Probabilidade de variação na próxima rodada após variação nas últimas {n} rodadas: {probabilidade:.2%}")
    print(f"Total de jogos testados para {n} rodadas: {total_jogos_testados}")


Probabilidade de variação na próxima rodada após variação nas últimas 1 rodadas: 64.71%
Total de jogos testados para 1 rodadas: 17
Probabilidade de variação na próxima rodada após variação nas últimas 2 rodadas: 63.64%
Total de jogos testados para 2 rodadas: 11
Probabilidade de variação na próxima rodada após variação nas últimas 3 rodadas: 70.00%
Total de jogos testados para 3 rodadas: 10
Probabilidade de variação na próxima rodada após variação nas últimas 4 rodadas: 75.00%
Total de jogos testados para 4 rodadas: 8
Probabilidade de variação na próxima rodada após variação nas últimas 5 rodadas: 75.00%
Total de jogos testados para 5 rodadas: 8
Probabilidade de variação na próxima rodada após variação nas últimas 6 rodadas: 66.67%
Total de jogos testados para 6 rodadas: 6
Probabilidade de variação na próxima rodada após variação nas últimas 7 rodadas: 66.67%
Total de jogos testados para 7 rodadas: 6
Probabilidade de variação na próxima rodada após variação nas últimas 8 rodadas: 60.00%